<a href="https://colab.research.google.com/github/tbeucler/2024_MLEES_Ebook/blob/main/Ayoub/Week_9_Generative_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (Exercise) Autoencoders, Generative Adversarial Networks, and Diffusion Models

![](https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Th%C3%A9%C3%A2tre_D%E2%80%99op%C3%A9ra_Spatial.png/920px-Th%C3%A9%C3%A2tre_D%E2%80%99op%C3%A9ra_Spatial.png)

Théâtre D'opéra Spatial, 2022 artwork created by Jason M. Allen with Midjourney